<h1> 개인과제 </h1>
<h2> 주피터랩에서 데이터를 입력받으면 3가지 DB에 동시에 CRUD가 되는 시스템을 만들어라.</h2>
<h3> 예) input을 이용하여 홍길동(엔터), 33(엔터), <전화번호> 111-2222(엔터), <주소> 해운대구 </h3>
<h3>입력하고, 이름을 고르고 수정하고, 삭제 가능하게 하라</h3>

In [60]:
def MYSQL():
    # MySQL에 연결
    import mysql.connector
    conn_sql = mysql.connector.connect(
            host="mysql", # MySQL 서버 주소
            user="root",
            password="1234",
            database="judb" #사용할 데이터 베이스
        )
    cursor_sql = conn_sql.cursor()
    # print("MySQL 연결성공")

In [48]:
def MONGODB():
    #  mongoDB 연결
    from pymongo import MongoClient
    client = MongoClient("mongodb://mongodb:27017")
    
    # 사용할 데이터베이스 선택
    db = client["mdb"]
    # print("사용할 데이터베이스 선택됨")
    
    # 사용할 컬렉션 선택
    collection = db["c1"]
    # print("사용할 컬렉션 선택됨")

In [49]:
def ORACLE():
    # oracle 연결
    import oracledb
        #데이터베이스 연결시도
    try:
        conn_ora = oracledb.connect(
            user="pknu",
            password="1234",
            dsn = "oracle/XEPDB1"
        )
        # print("Oracle 데이터베이스에 성공적으로 연결 되었습니다")
        cursor_ora = conn_ora.cursor()
        
    except Exception as e:
        print("Oracle 데이터베이스 연결 실패:",e)

In [50]:
def Create():
    sql = "INSERT INTO users(name, age, city) VALUES (%s, %s, %s)"
    values = (f"{inp_name}",inp_age, f"{inp_address}")
    cursor_sql.execute(sql, values)
    conn_sql.commit()
    print("MySQL 데이터 삽입 완료")

    user = {"name":f"{inp_name}","age":inp_age,"city":f"{inp_address}"}
    insert_result = collection.insert_one(user)
    print("mongoDB 데이터 삽입 완료")

    
    que = "INSERT INTO students (name,age,email) VALUES (:1,:2,:3)"
    values = (f"{inp_name}",inp_age, f"{inp_address}")
    cursor_ora.execute(que,values)
    conn_ora.commit()
    print("Oracle 데이터 삽입 완료")

In [65]:
def Read(inp_name):
    que = "SELECT * FROM users"
    filters = " WHERE name = %s"
    value = (f"{inp_name}",)
    cursor_sql.execute(que + filters,value)
    
    filtered = cursor_sql.fetchall()
    print("MySQL에서 조회한 정보 출력")
    for f in filtered:
        print(f)
    print("======================================================")
    user = collection.find_one({"name":f"{inp_name}"},{"_id":0})
    print("mongoDB에서 조회한 정보 출력")
    print(user)
    print("======================================================")
    que = "SELECT * FROM students WHERE name = :name"
    cursor_ora.execute(que, {'name':f"{inp_name}"})
    print("Oracle에서 조회한 정보 출력")
    users = cursor_ora.fetchall()
    for user in users:
        print(user)

In [52]:
def Update():
    if inp_data == "age":
        sql = "UPDATE users SET age = %s WHERE name = %s"
    elif inp_data == "name":
        sql = "UPDATE users SET name = %s WHERE name = %s"
    elif inp_data == "address":
        sql = "UPDATE users SET city = %s WHERE name = %s"
    else:
        print("잘못된 항목입니다.")
        return       
    values = (f"{inp_change}",f"{inp_name}")
    cursor_sql.execute(sql, values)
    conn_sql.commit()
    print("MySQL 수정완료")

    if inp_data == "address":
        update = collection.update_many({"name":f"{inp_name}"},{"$set":{"city":f"{inp_change}"}})
    elif inp_data == "name":
        update = collection.update_many({"name":f"{inp_name}"},{"$set":{"name":f"{inp_change}"}})
    elif inp_data == "age":
        update = collection.update_many({"name":f"{inp_name}"},{"$set":{"age":f"{inp_change}"}})
    else:
        print("잘못된 항목입니다.")
        return    
    print("mongoDB 수정완료")


    if inp_data == "address":
        que = "UPDATE students SET email = :1 WHERE name = :2"
    elif inp_data == "name":
        que = "UPDATE students SET name = :1 WHERE name = :2"
    elif inp_data == "age":
        que = "UPDATE students SET age = :1 WHERE name = :2"
    else:
        print("잘못된 항목입니다.")
        return    
        
    values = (f"{inp_change}",f"{inp_name}")
    cursor_ora.execute(que, values)
    conn_ora.commit()
    print("Oracle 수정완료")    

In [53]:
def Delete():
    que = "DELETE FROM users WHERE name = %s"
    cursor_sql.execute(que, (f"{inp_name}",))
    conn_sql.commit()
    print(f"MySQL {inp_name} 삭제완료")

    delete = collection.delete_one({"name":f"{inp_name}"})
    print(f"mongoDB {inp_name} 삭제완료")

    que = "DELETE FROM students WHERE name = :1"
    cursor_ora.execute(que, {1:f"{inp_name}"})
    conn_ora.commit()
    print(f"Oracle {inp_name} 삭제완료")

In [67]:
import mysql.connector
MYSQL(cursor_sql,conn_sql)
MONGODB()
ORACLE()
str= """
    1. 데이터 등록
    2. 데이터 읽기
    3. 데이터 수정
    4. 데이터 삭제
    5. 종료
    고르세요
    """
inp = 1

while True:
    print(str)
    inp = input("선택하세요:")
    print(inp,"고르셨네요")
    # 데이터 등록
    if inp == '1':
        inp_name = input("이름을 입력하세요:")
        inp_age = input("나이를 입력하세요:")
        inp_address = input("주소를 입력하세요:")
        Create()
        break
        
    if inp == '2':
        inp_name = input("찾고싶은 데이터의 name을 입력하세요")
        Read(inp_name)
        break
        
    if inp == '3':
        inp_update = input("변경하고 싶은 데이터의 name을 입력하세요")
        inp_data = input("변경하고자 하는 데이터의 종류를 입력하세요(name,age,address)")
        inp_change = input("변경할 데이터를 입력하세요")
        Update()
        break
        
    if inp == '4':
        inp_name = input("삭제하고 싶은 데이터의 name을 입력하세요")
        inp_choose = input("삭제하시겠습니까? (네, 아니요)")
        if inp_choose == "네":
            Delete()
        else:
            break
        
    if inp == '5':
        print("종료합니다.")
        break

NameError: name 'cursor_sql' is not defined